In [1]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.optim.lr_scheduler import OneCycleLR
from sklearn.preprocessing import StandardScaler
from utils import *
from torch.utils.tensorboard import SummaryWriter
import datetime
import time
#from model import InceptionTimePlus
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import roc_curve, roc_auc_score,precision_recall_fscore_support,f1_score,accuracy_score,precision_score,recall_score,balanced_accuracy_score

In [2]:
data = pd.read_csv('data_bin.csv')
data = data.drop('Unnamed: 0', axis=1)

In [3]:
# 特征和标签
X = data.drop('category', axis=1).values
y = data['category'].values
print(X.shape)
print(y.shape)

(2566, 411)
(2566,)


In [4]:
#standard the feature
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(1/2), random_state=42)

# resize the data adjust the CNN input (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(1539, 1, 411)
(1539,)
(513, 1, 411)
(513,)
(514, 1, 411)
(514,)


In [5]:
# 将数据转换为PyTorch张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

y_train_tensor  = F.one_hot(torch.tensor(y_train)).float()
y_val_tensor = F.one_hot(torch.tensor(y_val)).float()
y_test_tensor = F.one_hot(torch.tensor(y_test)).float()


# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
X_train_tensor.shape

torch.Size([1539, 1, 411])

In [6]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)
            # m.bias.data.zero_()

        elif isinstance(m, nn.BatchNorm1d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)


class CNN1D(nn.Module):
    def __init__(self, input_length, num_class=6):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5)          #[5, 17, 47]
        self.bn1 = nn.BatchNorm1d(32)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=17)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=47)
        self.bn3 = nn.BatchNorm1d(128)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(self._get_conv_output_size(input_length), 128)
        self.fc2 = nn.Linear(128, num_class)

    def _get_conv_output_size(self, input_length):
        size = input_length
        size = (size - 4) // 2  # conv1 and pool1
        size = (size - 16) // 2  # conv2 and pool2
        size = (size - 46) // 2  # conv3 and pool3
        return size * 128

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
CNN1D(411)

CNN1D(
  (conv1): Conv1d(1, 32, kernel_size=(5,), stride=(1,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(32, 64, kernel_size=(17,), stride=(1,))
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(64, 128, kernel_size=(47,), stride=(1,))
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2944, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=6, bias=True)
)

In [7]:
def train(model, criterion, optimizer, scheduler, n_epochs=5):
    best = 0
    train_step = 0
    losses = []

    # tensorboard log dir
    log_dir_name = "V1_6_binary"   # "V3_multiClass" 'V3_2_binary'
    log_dir = f"runs/{log_dir_name}_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    
    # init the TensorBoard
    writer = SummaryWriter(log_dir=log_dir)
    print(f"tensorboard --logdir={log_dir}")

    model.train()
    for epoch in range(n_epochs):
        since = time.time()
        train_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, targets = data
            
            inputs = inputs.cuda()
            targets = targets.cuda()
            
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, targets)
            loss.backward()

            # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()

            train_loss += loss.item()
            writer.add_scalar("train loss", loss.item(), train_step)
            train_step += 1
        
        epoch_duration = time.time() - since
        epoch_loss = train_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, duration: {epoch_duration:.2f}, loss: {epoch_loss:.6f}")

        losses.append(epoch_loss)

        # 验证模型
        model.eval()
        val_acc = val(model, val_loader)
        writer.add_scalar("val acc", val_acc, epoch + 1)
        if best < val_acc:
            best = val_acc
            torch.save(model, "weights/V1_6_binary.pth")

        model.train()
        #scheduler.step() # test_acc
    
    print('Finished Training')
    return model, losses

In [11]:
def val(model, val_dataloader):
    val_labels = []
    val_predictions = []
    with torch.no_grad():
        for i, (inputs, label) in enumerate(val_dataloader, 0):
            # images = images.to(device).half() # uncomment for half precision model
            inputs = inputs.cuda()
            label = label.cuda()
            outputs = model(inputs)

            val_labels.extend([label.cpu().numpy()])
            val_predictions.extend([torch.sigmoid(outputs).cpu().numpy()])
    
    val_labels = np.vstack(val_labels)
    val_predictions = np.vstack(val_predictions)

    val_predictions_prob = np.exp(val_predictions)/np.sum(np.exp(val_predictions),axis=1,keepdims=True)
    
    
    val_predictions = np.argmax(val_predictions,axis=1)
    val_labels = np.argmax(val_labels,axis=1)
    avg_score = accuracy_score(val_labels,val_predictions)
    p_marco = precision_score(val_labels,val_predictions,average='macro')
    r_marco = recall_score(val_labels,val_predictions,average='macro')
    
    print('Accuracy of the network on the test dataset: {}'.format(avg_score))
    print('p_marco of the network on the test dataset: {}'.format(p_marco))
    print('r_marco of the network on the test dataset: {}'.format(r_marco))
    return avg_score

In [23]:
learning_rate = 1e-5
num_classes = 2
epochs = 120

# model_ft = WaveLength(c_in=X.shape[1], c_out=6, nf=[47, 47, 47, 47])
model_ft = CNN1D(411, 2).cuda()
criterion = nn.BCEWithLogitsLoss() 
#nn.CrossEntropyLoss() # nn.BCEWithLogitsLoss()

#lrscheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, epochs=epochs, steps_per_epoch=len(train_dataloader))
# lrscheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=20, eta_min=1e-9)


optimizer = optim.AdamW(model_ft.parameters(), lr=learning_rate, weight_decay=learning_rate*0.5)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=7, verbose=True)
# parms_1x = [value for name, value in model.named_parameters()
#             if name not in ["fc.weight", "fc.bias"]]
# parms_10x = [value for name, value in model.named_parameters()
#             if name in ["fc.weight", "fc.bias"]]
# optimizer = optim.AdamW([{"params": parms_1x},
#                         {"params": parms_10x, 'lr': learning_rate * 10}], lr=learning_rate)

total_params = sum(p.numel() for p in model_ft.parameters())
print(f'Total parameters: {total_params}')  # 798406 参数大概合理
model_ft

Total parameters: 797890


/root/miniconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


CNN1D(
  (conv1): Conv1d(1, 32, kernel_size=(5,), stride=(1,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(32, 64, kernel_size=(17,), stride=(1,))
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(64, 128, kernel_size=(47,), stride=(1,))
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2944, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [24]:
model_ft, training_losses = train(model_ft, criterion, optimizer, lrscheduler, n_epochs=epochs)

tensorboard --logdir=runs/V1_6_binary_20240613-184618
Epoch 1, duration: 0.33, loss: 0.688778
Accuracy of the network on the test dataset: 0.5107212475633528
p_marco of the network on the test dataset: 0.48311048557353975
r_marco of the network on the test dataset: 0.49415364940166256
Epoch 2, duration: 0.32, loss: 0.676804
Accuracy of the network on the test dataset: 0.5692007797270955
p_marco of the network on the test dataset: 0.7735655737704918
r_marco of the network on the test dataset: 0.5508130081300813
Epoch 3, duration: 0.32, loss: 0.660906
Accuracy of the network on the test dataset: 0.6705653021442495
p_marco of the network on the test dataset: 0.6730605285592498
r_marco of the network on the test dataset: 0.6668950397369142
Epoch 4, duration: 0.32, loss: 0.641272
Accuracy of the network on the test dataset: 0.6744639376218323
p_marco of the network on the test dataset: 0.6948353054045411
r_marco of the network on the test dataset: 0.6669635516579885
Epoch 5, duration: 0.32,

In [14]:
# model = WaveLength(c_in=411, c_out=6, nf=[47, 128, 128, 47])
model = torch.load('weights/v1_6_binary.pth')
model.eval()  # 切换模型到评估模式
print("Model loaded successfully.")

Model loaded successfully.


In [18]:
test_labels = []
test_predictions = []
with torch.no_grad():
    for i, (inputs, label) in enumerate(test_loader, 0):
        # images = images.to(device).half() # uncomment for half precision model
        inputs = inputs.cuda()
        label = label.cuda()
        outputs = model(inputs)

        test_labels.extend([label.cpu().numpy()])
        test_predictions.extend([torch.sigmoid(outputs).cpu().numpy()])

test_labels = np.vstack(test_labels)
test_predictions = np.vstack(test_predictions)

test_predictions_prob = np.exp(test_predictions)/np.sum(np.exp(test_predictions),axis=1,keepdims=True)


test_predictions = np.argmax(test_predictions,axis=1)
test_labels = np.argmax(test_labels,axis=1)

avg_score = accuracy_score(test_labels, test_predictions)
p_marco = precision_score(test_labels,test_predictions,average='macro')
r_marco = recall_score(test_labels,test_predictions,average='macro')
f1_marco = f1_score(test_labels,test_predictions,average='macro')
print("Average accuracy: {:.4f}".format(avg_score))
print("f1_marco: {:.4f}".format(f1_marco))
print("p_marco: {:.4f}".format(p_marco))
print("r_marco: {:.4f}".format(r_marco))

Average accuracy: 0.9903
f1_marco: 0.9902
p_marco: 0.9904
r_marco: 0.9901


In [ ]:
# [1 1 2 2 3 3 4 4 5 5 0 0]
# [1 0 2 2 3 5 5 5 5 5 0 0]